In [ ]:
#What is the reason behind Heart disease

In [ ]:
# Introduction
'''
Heart disease describes a range of conditions that affect your heart. Diseases under the heart disease umbrella include blood vessel diseases, such as coronary artery disease; heart rhythm problems (arrhythmias); and heart defects you're born with (congenital heart defects), among others.

The term "heart disease" is often used interchangeably with the term "cardiovascular disease." Cardiovascular disease generally refers to conditions that involve narrowed or blocked blood vessels that can lead to a heart attack, chest pain (angina) or stroke. Other heart conditions, such as those that affect your heart's muscle, valves or rhythm, also are considered forms of heart disease.
'''

In [3]:
#Import Datas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns #for plotting
import os

In [4]:
np.random.seed(101) #ensure reproducibility
pd.options.mode.chained_assignment = None  #hide any pandas warnings

In [6]:
cwd = os.getcwd()
DataPath = cwd + '/Data/heart.csv'
print("Current working directory",cwd)
print("Data Directory",DataPath)

Current working directory /home/ibrahim/Repos/Heart-Disease
Data Directory /home/ibrahim/Repos/Heart-Disease/Data/heart.csv


In [7]:
df = pd.read_csv(DataPath)

In [8]:
df.head(3)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
